<a href="https://colab.research.google.com/github/mafemunoz/Trabajo-IA-2025-2/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Carga el archivo kaggle.json
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!python -m pip install -qq kaggle
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia

Saving kaggle (2).json to kaggle (2).json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
unzip:  cannot find or open udea-ai-4-eng-20252-pruebas-saber-pro-colombia, udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip or udea-ai-4-eng-20252-pruebas-saber-pro-colombia.ZIP.


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier

In [4]:
test_df = pd.read_csv('/content/test.csv', encoding= 'utf-8')
df = pd.read_csv('/content/train.csv', encoding= 'utf-8')

**Preprocesado**

El preprocesamiento que ofreció mejores resultados fue el siguiente: se mantuvo la limpieza de la columna "programa" como en los modelos anteriores, pero a diferencia de estos, en este caso se imputaron los valores nulos de las columnas categóricas con la categoría "desconocido", mientras que en las columnas numéricas se utilizaron los promedios. En general, el resto del proceso se conservó igual que en los otros archivos alojados en GitHub.

Las transformaciones adicionales realizadas incluyen:

Eliminación de las columnas "E_PRIVADO_LIBERTAD" y una columna repetida de "F_TIENEINTERNET.1".

Renombramiento de los índices para facilitar la manipulación de los datos.

Conversión de los valores booleanos a valores binarios.

Imputación de los datos categóricos faltantes utilizando la moda de cada categoría.

Codificación One-Hot de todas las columnas categóricas.

Transformación de la columna "rendimiento_global" a valores ordinales.



In [5]:
def Preprocesar(df):
    def obtener_primera_frase(cadena):
        return cadena.split(' ', 1)[0]

    def quitar_tildes(palabra):
        return re.sub(r'[ÁÉÍÓÚ]', lambda x: {'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}[x.group()], palabra)

    df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(obtener_primera_frase)
    df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(quitar_tildes)

    reemplazos = {
        'ADMINISTRACION': ['ADMINISTRACION', 'ADMINISTRACI¿N', 'ADMINISTRACIÒN', 'ADMINISTRACIÓN', 'ADMINSITRACION', 'ADMINSITRACIÓN'],
        'ANTROPOLOGIA': ['ANTROPOLOGIA', 'ANTROPOLOGÍA'],
        'ARTES': ['ARTE', 'ARTES', 'BELLAS', 'ANIMACION'],
        'BIOINGENIERIA': ['BIOINGENIERIA', 'BIOINGENIERÍA'],
        'BIOLOGIA': ['BIOLOGIA', 'BIOLOGÍA'],
        'COMUNICACION': ['COMUNICACION', 'COMUNICACIONES', 'COMUNICACI¿N', 'COMUNICACIÓN', 'COMUNICACIÖN'],
        'CONSTRUCCION': ['CONSTRUCCION', 'CONSTRUCCIONES'],
        'CONTADURIA': ['CONTADURIA', 'CONTADURÍA'],
        'DISEÑO': ['DISENO', 'DISE¿O', 'DISE¿¿O', 'DISEÑO'],
        'ECOLOGIA': ['ECOLOGIA', 'ECOLOGÍA'],
        'ECONOMIA': ['ECONOM¿A'],
        'GEOLOGIA': ['GEOLOG¿A'],
        'GERENCIA': ['GERONTOLOGIA'],
        'INGENIERIA': ['INGENIER¿A', 'INGENIERÌA', 'INGENIIERIA', 'AGROINDUSTRIA'],
        'INSTRUMENTACION': ['INTRUMENTACION'],
        'MATEMATICAS': ['MATEMATICA'],
        'MARKETING': ['MERCADEO', 'MERCADOLOGIA'],
        'PSICOLOGIA': ['PSICOLOGA'],
        'QUIMICA': ['QUIMICA.', 'QU¿MICA'],
        'MUSICA': ['MUSICA-'],
        'TERAPIAS': ['TERAPIA', 'TERAPIAS'],
        'GESTION': ['GESTI¿N'],
        'NEGOCIOS': ['3°'],
        'DEPORTE': ['ACTIVIDAD'],
        'LICENCIATURA': ['Licenciatura', 'LICENC.EN'],
        'LENGUAS': ['LENGUAJES']
    }

    for key, values in reemplazos.items():
        df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].replace(values, key)

    # Eliminar columnas innecesarias
    df = df.drop(labels =['E_PRIVADO_LIBERTAD','F_TIENEINTERNET.1', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'],axis= 1)

    # Renombrar columnas si es necesario
    if 'RENDIMIENTO_GLOBAL' in df.columns:
        df.columns = ['ID', 'PERIODO', 'PROGRAMA', 'DEPARTAMENTO', 'VALOR_MATRICULA', 'HORAS_TRABAJA', 'ESTRATO', 'TIENE_INTERNET', 'EDUCACION_PADRE', 'TIENE_LAVADORA', 'TIENE_AUTO', 'PAGA_MATRICULA', 'TIENE_COMPUTADOR', 'EDUCACION_MADRE', 'RENDIMIENTO_GLOBAL']
    else:
        df.columns = ['ID', 'PERIODO', 'PROGRAMA', 'DEPARTAMENTO', 'VALOR_MATRICULA', 'HORAS_TRABAJA', 'ESTRATO', 'TIENE_INTERNET', 'EDUCACION_PADRE', 'TIENE_LAVADORA', 'TIENE_AUTO', 'PAGA_MATRICULA', 'TIENE_COMPUTADOR', 'EDUCACION_MADRE']

    # Convertir valores de 'Si'/'No' a binario
    def convertir_num(valor):
        if valor == 'Si':
            return 1
        elif valor == 'No':
            return 0
        else:
            return valor

    df = df.applymap(convertir_num)

    # Imputar valores nulos en columnas categóricas con 'Desconocido'
    categorical_columns = ["VALOR_MATRICULA",'PERIODO', 'ESTRATO',"PROGRAMA", "DEPARTAMENTO", "EDUCACION_PADRE", "EDUCACION_MADRE"]
    for col in categorical_columns:
        df[col].fillna('Desconocido', inplace=True)

    # Imputar valores nulos en columnas numéricas con la media
    numerical_columns = df.select_dtypes(include=[np.number]).columns
    for col in numerical_columns:
        df[col].fillna(df[col].mean(), inplace=True)

    # Convertir a categorías las columnas necesarias
    df[categorical_columns] = df[categorical_columns].astype("category")

    # Reemplazar valores de rendimiento
    valores_rendimiento = {'alto': 4, 'medio-alto': 3, 'medio-bajo': 2, 'bajo': 1}
    df = df.replace(valores_rendimiento)

    # Aplicar One-Hot Encoding a columnas categóricas específicas
    encoder = OneHotEncoder(sparse_output=False)

    encoded_columns = encoder.fit_transform(df[['HORAS_TRABAJA', 'PERIODO', 'VALOR_MATRICULA', 'PROGRAMA', 'DEPARTAMENTO', 'ESTRATO', 'EDUCACION_PADRE', 'EDUCACION_MADRE']])

    feature_names = encoder.get_feature_names_out(['HORAS_TRABAJA', 'PERIODO', 'VALOR_MATRICULA', 'PROGRAMA', 'DEPARTAMENTO', 'ESTRATO', 'EDUCACION_PADRE', 'EDUCACION_MADRE'])

    one_hot_encoded_df = pd.DataFrame(encoded_columns, columns=feature_names)

    # Eliminar columnas originales que fueron codificadas
    df.drop(labels=['HORAS_TRABAJA', 'PERIODO', 'VALOR_MATRICULA', 'PROGRAMA', 'DEPARTAMENTO', 'ESTRATO', 'EDUCACION_PADRE', 'EDUCACION_MADRE'], axis=1, inplace=True)

    # Concatenar el DataFrame original con el DataFrame codificado
    df = pd.concat([df, one_hot_encoded_df], axis=1)

    # Verificar si hay valores nulos en el DataFrame final
    print(df.isnull().sum())

    return df

In [6]:
df = Preprocesar(df)
test_df = Preprocesar(test_df)

/tmp/ipython-input-564144349.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(convertir_num)
/tmp/ipython-input-564144349.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('Desconocido', inplace=True)
/tmp/ipython-input-564144349.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the 

ID                                                      0
TIENE_INTERNET                                          0
TIENE_LAVADORA                                          0
TIENE_AUTO                                              0
PAGA_MATRICULA                                          0
                                                       ..
EDUCACION_MADRE_Primaria incompleta                     0
EDUCACION_MADRE_Secundaria (Bachillerato) completa      0
EDUCACION_MADRE_Secundaria (Bachillerato) incompleta    0
EDUCACION_MADRE_Técnica o tecnológica completa          0
EDUCACION_MADRE_Técnica o tecnológica incompleta        0
Length: 204, dtype: int64


/tmp/ipython-input-564144349.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(convertir_num)
/tmp/ipython-input-564144349.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('Desconocido', inplace=True)
/tmp/ipython-input-564144349.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the 

ID                                                      0
TIENE_INTERNET                                          0
TIENE_LAVADORA                                          0
TIENE_AUTO                                              0
PAGA_MATRICULA                                          0
                                                       ..
EDUCACION_MADRE_Primaria incompleta                     0
EDUCACION_MADRE_Secundaria (Bachillerato) completa      0
EDUCACION_MADRE_Secundaria (Bachillerato) incompleta    0
EDUCACION_MADRE_Técnica o tecnológica completa          0
EDUCACION_MADRE_Técnica o tecnológica incompleta        0
Length: 203, dtype: int64


**Modelo CatBoostClassifier**

el modelo que mejores resultados nos arrojo es este llamado CatBoostClassifier el cual al igual que el random forest esta basado en arboles de decision, sin embargo, a diferencia del random forest que crea los arboles en paralelo este modelo lo hace de manera secuencial, teniendo oportunidad de corregir los errores de los arboles anteriores en cada iteracion.

Este modelo ademas de arrojar resultados excepcionales en cuanto a la clasificación tambien nos presento tiempos de ejecucción excelentes de manera que pude incluso agregarle una gran cantidad de iteracciones a modo de prueba para ver si mejoraba los resultados.

Finalmente el mejor resultado que pude encontrar la fase de entrenamiento fue este. Si bien luego disminuye el acuraccy para la fase de predict creemos que esto se puede deber a un ligero sobreajuste y al hecho de que el dataset de test tiene información que el modelo no vio en etapa de aprendizaje.

In [8]:
X = df.drop(labels =["RENDIMIENTO_GLOBAL",'ID'], axis=1)  # Características
y = df["RENDIMIENTO_GLOBAL"]  # Variable objetivo

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state = 42)

# Crear el modelo CatBoost
catboost_model = CatBoostClassifier(iterations=2000, learning_rate=0.1, depth=6, loss_function='MultiClass')

# Entrenar el modelo
catboost_model.fit(X_train, y_train)

# Evaluar el modelo
accuracy = catboost_model.score(X_test, y_test)
print("Accuracy:", accuracy)

0:	learn: 1.3694010	total: 341ms	remaining: 11m 22s
1:	learn: 1.3558743	total: 675ms	remaining: 11m 14s
2:	learn: 1.3453566	total: 1s	remaining: 11m 9s
3:	learn: 1.3361408	total: 1.39s	remaining: 11m 34s
4:	learn: 1.3281485	total: 2.01s	remaining: 13m 21s
5:	learn: 1.3215930	total: 2.58s	remaining: 14m 19s
6:	learn: 1.3159744	total: 3.22s	remaining: 15m 17s
7:	learn: 1.3109502	total: 3.74s	remaining: 15m 31s
8:	learn: 1.3067082	total: 4.23s	remaining: 15m 35s
9:	learn: 1.3028659	total: 4.53s	remaining: 15m 1s
10:	learn: 1.2993714	total: 4.87s	remaining: 14m 40s
11:	learn: 1.2961281	total: 5.23s	remaining: 14m 27s
12:	learn: 1.2929949	total: 5.55s	remaining: 14m 8s
13:	learn: 1.2902609	total: 5.86s	remaining: 13m 51s
14:	learn: 1.2877290	total: 6.17s	remaining: 13m 36s
15:	learn: 1.2852381	total: 6.49s	remaining: 13m 24s
16:	learn: 1.2829693	total: 6.81s	remaining: 13m 14s
17:	learn: 1.2810612	total: 7.16s	remaining: 13m 8s
18:	learn: 1.2794211	total: 7.46s	remaining: 12m 57s
19:	learn:

In [9]:
predictions = catboost_model.predict(test_df.drop('ID', axis=1))

predictions = predictions.flatten()

submit = pd.DataFrame({'ID': test_df['ID'], 'RENDIMIENTO_GLOBAL': predictions})

submit['RENDIMIENTO_GLOBAL'] = submit['RENDIMIENTO_GLOBAL'].replace({
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
})

submit.to_csv('submission.csv', index=False)